In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### **Algoritmos a probar**

**Linear**
* Logistic Regression
* Linear SVC (regularized - unregularized)

**Non-linear**
* Poly SVC
* rbf SVC

**otros**
* naïve Bayes
* Nearest Neighbors Classification
* Stochastic Gradient Descent
* Gaussian Process Classification
* Decision Trees

**Ensambles**
* Random Forests
* AdaBoost
* Gradient Tree Boosting

**Neural Network**
* Multi-layer Perceptron

### Pre-procesamiento de datos

In [2]:
# Cargo datos parte1
avisos_parte1 =  pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
postulaciones_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
vistas_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
edadYGenero_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
educacion_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

# Cargo datos hasta 15_4
avisos_hasta15_4 =  pd.read_csv('./data/Fiuba hasta 15 Abril/fiuba_6_avisos_detalle.csv')
postulaciones_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_4_postulaciones.csv')
vistas_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_3_vistas.csv')
edadYGenero_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')
educacion_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_1_postulantes_educacion.csv')

# Concateno la info
avisos = pd.concat([avisos_parte1, avisos_hasta15_4])
postulaciones = pd.concat([postulaciones_parte1, postulaciones_hasta15_4])
vistas = pd.concat([vistas_parte1, vistas_hasta15_4])
edadYGenero = pd.concat([edadYGenero_parte1, edadYGenero_hasta15_4])
educacion = pd.concat([educacion_parte1, educacion_hasta15_4])

# Elimino duplicados
avisos.drop_duplicates(inplace=True)
postulaciones.drop_duplicates(inplace=True)
vistas.drop_duplicates(inplace=True)
edadYGenero.drop_duplicates(inplace=True)
educacion.drop_duplicates(inplace=True)

# Remuevo columnas innecesarias
avisos.drop(columns=['idpais','ciudad','mapacalle'], inplace=True)

# Renombro columnas
educacion.rename(columns={'nombre':'nivel'}, inplace=True)
vistas.rename(columns={'idAviso':'idaviso','timestamp':'fechavista'}, inplace=True)